In [ ]:
# convert_to_tiles.py produces tiles which slightly off
# instead of retiling everything we try to stretch/fit them to grid tiles

In [84]:
import fiona
import gdal
import shapely.geometry
from shapely.geometry import mapping
import shapely as sl
import glob
from osgeo import gdal
import subprocess
import os.path

In [87]:
def clip_raster(in_path, out_path, shp_path, bounds, dx, dy):
    # read scale from original raster
    cmd = u'gdalwarp -co BIGTIFF=YES -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -tr {3} {4} -te {5} {6} {7} {8} -dstnodata -99999 -q -cutline {0} -of GTiff {1} {2}'.format(shp_path, in_path, out_path, dx, dy, bounds[0], bounds[1], bounds[2], bounds[3])
    subprocess.check_call(cmd, shell=True)

    pass

def write_shp(geom, shp_path, id):
    schema = { 'geometry': 'Polygon', 'properties': {'id': 'int'}, }

    with fiona.open(shp_path, 'w', 'ESRI Shapefile', schema) as c:
        c.write({'geometry': mapping(geom), 'properties': {'id': id}})


In [ ]:
pixel_size = 0.000833333333333
 
grid_file_path = r'../shared/grid-90m-exact.shp'
with fiona.collection(grid_file_path, 'r') as input:
    for f in input:
        geom = sl.geometry.shape(f['geometry'])
        id = f['id']
        print(f['id'].zfill(3))
        
        in_path = r'../output/tiles-90/hand-556/HAND_556_' + id.zfill(3) + '.0.tif'
        out_path = r'../output/tiles-90/hand-556/fixed/HAND_556_' + id.zfill(3) + '.0.tif'
        
        if os.path.exists(in_path):
            cell_path = 'cell.shp'
            write_shp(geom, cell_path, id)
            clip_raster(in_path, out_path, cell_path, geom.bounds, pixel_size, -pixel_size)

000
001
002
003
004
005
006
007
008